In [1]:
# Import the packages we'll use

import numpy as np
import os, glob, csv

import tensorflow as tf
# # librosa is a widely-used audio processing library
import librosa

import sklearn
import os
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torchaudio
from torchaudio import transforms as t
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as nnF

# # for plotting
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
os.getcwd()

'/Users/zayyanmatin/Desktop/Masters/DLAM'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
os.listdir()

['non project stuff',
 'Pytorch_Audio_Classifier_Procedure_Raw-Copy1.ipynb',
 'Pytorch_Audio_Classifier_Procedure-Testing.ipynb',
 'Pytorch_Audio_Classifier_Procedure_Raw-Copy3.ipynb',
 'Untitled1.ipynb',
 'test1.rtf',
 'Pytorch_Audio_Classifier_Procedure_Mel-Copy1.ipynb',
 '.DS_Store',
 'bgBinary.csv',
 'Pytorch_Audio_Classifier_Procedure_Raw-Copy2.ipynb',
 'Untitled.ipynb',
 'bird_bg04.wav',
 'bg_class.csv',
 'bgBinary 2.numbers',
 'other projects',
 'bgBinary_Crowd.csv',
 'Training_Validation_Maker.ipynb',
 'bgBinary.numbers',
 'env',
 'WaveNet-master ',
 '.ipynb_checkpoints',
 'tensorflow-wavenet-master',
 'Isolated_urban_sound_database',
 'Pytorch_Audio_Classifier_Procedure_Raw.ipynb',
 'Pytorch_Audio_Classifier_Procedure_Mel.ipynb']

In [5]:
csvData = pd.read_csv('./bgBinary_Crowd.csv')
print(len(csvData))
print(csvData.iloc[80,2])

162
0


In [6]:
class UrbanSoundDataset(Dataset):
#rapper for the UrbanSound8K dataset
    # Argument List
    #  path to the UrbanSound8K csv file
    #  path to the UrbanSound8K audio files
    #  list of folders to use in the dataset
    
    def __init__(self, csv_path, file_path, folderList):
        csvData = pd.read_csv(csv_path)
        #initialize lists to hold file names, labels, and folder numbers
        self.file_names = []
        self.labels = []
        self.folders = []
        #loop through the csv entries and only add entries from folders in the folder list
        for i in range(0,len(csvData)):
            if csvData.iloc[i, 3] in folderList:
                self.file_names.append(csvData.iloc[i, 0])
                self.labels.append(csvData.iloc[i, 2])
                self.folders.append(csvData.iloc[i, 3])
                
        self.file_path = file_path
        self.mixer = torch.mean #UrbanSound8K uses two channels, this will convert them to one
        self.folderList = folderList
        
    def __getitem__(self, index):
        #format the file path and load the file
        path = self.file_path + "fold" + str(self.folders[index]) + "/" + self.file_names[index]
        sound = torchaudio.load(path, out = None, normalization = True)
        #load returns a tensor with the sound data and the sampling frequency (44.1kHz for UrbanSound8K)
        soundData = torch.mean(sound[0], 0, keepdim=False, out=None)
        soundData = soundData.unsqueeze_(-1)
        #downsample the audio to ~8kHz
        tempData = torch.zeros([160000, 1]) #tempData accounts for audio clips that are too short
        if soundData.numel() < 160000:
            tempData[:soundData.numel()] = soundData[:]
        else:
            tempData[:] = soundData[:160000]
        
        soundData = tempData
        soundFormatted = torch.zeros([32000, 1])
        soundFormatted[:32000] = soundData[::5] #take every fifth sample of soundData
        soundFormatted = soundFormatted.permute(1, 0)
        return soundFormatted, self.labels[index]
    
    def __len__(self):
        return len(self.file_names)

    
csv_path = './bgBinary_Crowd.csv'
file_path = './Isolated_urban_sound_database/'

train_set = UrbanSoundDataset(csv_path, file_path, range(0,4))
test_set = UrbanSoundDataset(csv_path, file_path, [4])
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 32, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 32, shuffle = True, **kwargs)

Train set size: 130
Test set size: 32


In [7]:
train_set

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, 80, 4)
        self.bn1 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(128, 128, 3)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(128, 256, 3)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(256, 512, 3)
        self.bn4 = nn.BatchNorm1d(512)
        self.pool4 = nn.MaxPool1d(4)
        self.avgPool = nn.AvgPool1d(30) #input should be 512x30 so this outputs a 512x1
        self.fc1 = nn.Linear(512, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = self.avgPool(x)
        x = x.permute(0, 2, 1) #change the 512x1 to 1x512
        x = self.fc1(x)
        return F.log_softmax(x, dim = 2)

model = Net()
model.to(device)
print(model)

Net(
  (conv1): Conv1d(1, 128, kernel_size=(80,), stride=(4,))
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(256, 512, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (avgPool): AvgPool1d(kernel_size=(30,), stride=(30,), 

In [25]:
optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)


In [26]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
        output = output.permute(1, 0, 2) #original output dimensions are batchSizex1x10 
        loss = F.nll_loss(output[0], target) #the loss functions expects a batchSizex10 input
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))
        gLoss = loss
        
    if loss <= 0.00010:
        
        return 1
    
    else:
        return 0

In [27]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        output = output.permute(1, 0, 2)
        pred = output.max(2)[1] # get the index of the max log-probability
        correct += pred.eq(target).cpu().sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [28]:
log_interval = 2
for epoch in range(1, 27):
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
    
    train(model, epoch)
    test(model, epoch)
#     if train(model, epoch) == 1:
#         break
    

Train Epoch: 1 [0/130 (0%)]	Loss: 0.613367
Train Epoch: 1 [64/130 (40%)]	Loss: 0.872276
Train Epoch: 1 [8/130 (80%)]	Loss: 0.000950

Test set: Accuracy: 27/32 (84%)

Train Epoch: 2 [0/130 (0%)]	Loss: 0.118095
Train Epoch: 2 [64/130 (40%)]	Loss: 0.370967
Train Epoch: 2 [8/130 (80%)]	Loss: 0.000188

Test set: Accuracy: 27/32 (84%)

Train Epoch: 3 [0/130 (0%)]	Loss: 0.157373
Train Epoch: 3 [64/130 (40%)]	Loss: 0.357671
Train Epoch: 3 [8/130 (80%)]	Loss: 0.027621

Test set: Accuracy: 27/32 (84%)

Train Epoch: 4 [0/130 (0%)]	Loss: 0.155964
Train Epoch: 4 [64/130 (40%)]	Loss: 0.239620
Train Epoch: 4 [8/130 (80%)]	Loss: 0.001803

Test set: Accuracy: 27/32 (84%)

Train Epoch: 5 [0/130 (0%)]	Loss: 0.234977
Train Epoch: 5 [64/130 (40%)]	Loss: 0.322073
Train Epoch: 5 [8/130 (80%)]	Loss: 1.748919

Test set: Accuracy: 27/32 (84%)

Train Epoch: 6 [0/130 (0%)]	Loss: 0.137097
Train Epoch: 6 [64/130 (40%)]	Loss: 0.657765
Train Epoch: 6 [8/130 (80%)]	Loss: 2.018602

Test set: Accuracy: 27/32 (84%)

Trai

In [23]:
torch.save(model.state_dict(), 'raw.pt')